# Sliding Window

Solutions to sliding window Leetcode problems

In [2]:
from typing import List

# Problem One: Best Time to Buy and Sell Stock (Easy)

[Leetcode #121](https://leetcode.com/problems/best-time-to-buy-and-sell-stock/)

Keep a sliding window initialized at `left = 0, right = 1`. Also keep track of current max profit. If current window results in a profit, compare and possible update max proft. Else, shift `left` to `right` to try and find a profit. Shift `right` by one after every step, and loop while `right < N` where `N` is the length of `prices`.

In [3]:
def maxProfit(prices: List[int]) -> int:
        left, right = 0, 1
        maxP = 0

        while right < len(prices):
            if prices[left] < prices[right]:
                profit = prices[right] - prices[left]
                maxP = max(maxP, profit)
            else:
                left = right
            
            right += 1
        
        return maxP

# Problem Two: Longest Substring Without Repeating Characters (Medium)

[Leetcode #3](https://leetcode.com/problems/longest-substring-without-repeating-characters/)

* Initialize `left` and `right` pointers both to `0`. Initialize `longest` to `0`. Initialize a `defaultdict` (all keys will by default be initialized to 0 so we don't ahve to check for existance) called `counts`.

* While `right` < `N`:
    * Increment `counts[s[right]]`
    * While `counts[s[right]] > 1` (i.e. there is a repeating character), decrease window from left side. First, decrement `counts[s[left]]`, then increment `left` pointer
    * Update `longest` to `max(longest, right - left + 1)`
    * Increment `right` pointer

* Return `longest`
    

In [4]:
def lengthOfLongestSubstring(s: str) -> int:
        N = len(s)

        left, right = 0, 0
        
        counts = collections.defaultdict(int)
        longest = 0

        while right < N:
            counts[s[right]] += 1
            while counts[s[right]] > 1:
                counts[s[left]] -= 1
                left += 1
            
            longest = max(longest, right - left + 1)
            right += 1
        
        return longest

# Problem Three: Longest Repeating Character Replacement (Medium)

[Leetcode #424](https://leetcode.com/problems/longest-repeating-character-replacement/)

* Initialize `left` and `right` to `0`. Initialize `counts` to an array of 26 `0`'s (there are 26 uppercase english letters). Initialize `longest` and `maxFreq` to 0.

* While `right < N` where `N` is the length of `s`:
    * Increment `counts[s[right]]`
    * Update `maxFreq` to `max(maxFreq, counts[s[right]])`
    * While the difference of window length (`right - left + 1`) and `maxFreq` is greater than `k` (i.e. we will need more than `k` replacements to make this window have only one character, i.e. the window is invalid), decrease `counts[s[left]]` and decrease window size from left by incrementing `left` pointer
    * Update `longest` to `max(longest, right - left + 1)`
    * Increment `right`
  
* Return `longest`

In [5]:
def characterReplacement(s: str, k: int) -> int:
        N = len(s)

        left, right = 0, 0
        counts = [0] * 26
        longest, maxFreq = 0, 0

        # Function for converting uppercase english letter to int index
        ind = lambda x: ord(x) - ord('A')

        while right < N:
            c = s[right]
            counts[ind(c)] += 1
            maxFreq = max(maxFreq, counts[ind(c)])

            # While window length - max in counts > k
            while right - left + 1 - maxFreq > k:
                # Decrease window from left side
                counts[ind(s[left])] -= 1
                left += 1
            
            # Update longest
            longest = max(longest, right - left + 1)
            # Increase window to the right
            right += 1
        
        return longest

# Problem Four: Permutation in String (Medium)

[Leetcode #567](https://leetcode.com/problems/permutation-in-string/)

**Key point**: Can check if string `a` is permutation of string `b` if counts of characters are the same. First count the characters in the substring, `s1`. This doesn't change. Next, keep a sliding window of size `len(s1)`. Compare the counts of the substring of `s2` represented by the sliding window and the counts in `s1`. If they are the same, return `True`, else slide the window to the right. Note, only the counts of characters and `left` and `right` in `s2` need to updated. If we have looked at all substrings in `s2` (`right == len(s2) - 1`), a permutation of `s1` did not exist withing `s2`, thus return `False`.

In [7]:
def checkInclusion(s1: str, s2: str) -> bool:
        M, N = len(s1), len(s2)

        # Function for converting uppercase english letter to int index
        ind = lambda c: ord(c) - ord('a')
        
        # Function for checking equality of the lists
        def eq(a, b):
            for i in range(26):
                if not a[i] == b[i]:
                    return False
            return True

        # If the substring is bigger than the string we are searching in, return False
        if M > N:
            return False

        # Initialize count dicts
        countsM, countsN = [0] * 26, [0] * 26

        # Fill counts for substring and first M characers of search string
        for i in range(M):
            countsM[ind(s1[i])] += 1
            countsN[ind(s2[i])] += 1

        # Initialize pointers
        left, right = 0, M

        while right < N:
            # Check for equality of count dicts
            if eq(countsM, countsN):
                return True
            
            # Increment counts of right pointer
            countsN[ind(s2[right])] += 1
            # Increment right pointer
            right += 1
            # Decrement counts of left pointer
            countsN[ind(s2[left])] -= 1
            # Increment left pointer
            left += 1
        
        # Final check for equality of count dicts
        if eq(countsM, countsN):
            return True
        
        return False

# Problem Five: Minimum Window Substring *(Hard)*

[Leetcode #76](https://leetcode.com/problems/minimum-window-substring/)

* **Complications**: Unlike question above, we do not know the size of the answer, so we cannot just slide around a constant size window.

* **Trick**: Instead of checking the entire `counts` dicts each time, we keep track of how many "conditions" (i.e. letters in `t`) we *have* satisfied, and compare with how many we *need* to satisify (this stays constant, and is equal to the number of distinct letters in `t`). To ensure `O(N + M)` runtime, we only check if adding or removing the character when sliding the window (anytime you change `left` or `right` pointers) affects the number of conditions satisfied.

Steps:

* Initialize `counts` and `subCounts` dicts

* Calculate `subCounts`

* Initialize `left` and `right` to `0`

* Initialize `have` to `0` and `need` to the number of distinct characters in `t`. I achieved this by filtering list of `subCounts` for all values greater than `0`, then taking the length of that list.

* Initialize `res` to `[-1, -1]` (this will contain the two pointers representing the min window) and `minLen` to `N + 1` (the minimum window size is guaranteed to be at most `N`. If no such window was found, the value will still be `N + 1` and we can return `""` as instructed)

* while `right < N`:
    * Increment `counts` of character at `right`
    * If `subCounts` of this character was not `0`, it is one of our "conditions". If we have met this condition, meaning the value for this character is the same in `counts` and `subCounts`, increment `have` counter
    * while `have == need`:
        * If this window is the minimum window so far, update `res` and `minLen`
        * Shrink window from the left. First decrease `counts` of left character. Next, check if this results in a condition not being met. This will be the case if the value for this character in `counts` is strictly less than that of `subCounts`. If so, decrement `have` counter. Increment `left` counter
     * Increment `right`

* If `minLen == N + 1`, no such window was found so return `""`. Else, return the substring of s bounded by `[res[0], res[1] + 1]`

In [12]:
def minWindow(s: str, t: str) -> str:
        # Initialize M and N
        N, M = len(s), len(t)

        if N < M or t == "":
            return ""

        # Function for converting english letter to index (0-25 for a-z, 26-51 for A-Z)
        ind = lambda c: ord(c) - ord('a') if ord(c) >= ord('a') else 26 + ord(c) - ord('A')

        # Initialize counts arrays
        counts = [0] * 52
        subCounts = [0] * 52

        # Calculate counts of substring
        for i in range(M):
            subCounts[ind(t[i])] += 1

        # Initialize pointers
        left, right = 0, 0

        # Initialize number of conditions satisifed / need to satisfy
        have, need = 0, len(list(filter(lambda x: x > 0, subCounts)))

        # Initialize minLength
        res, minLen = [-1, -1], N + 1

        while right < N:
            c = s[right]
            counts[ind(c)] += 1

            # If we just satisfied a condition for a required character (subCounts != 0 for character)
            if counts[ind(c)] == subCounts[ind(c)] and not subCounts[ind(c)] == 0:
                have += 1
            
            while have == need:
                # Update result and min length
                if (right - left + 1) < minLen:
                    res = [left, right]
                    minLen = right - left + 1
                
                # Shrink window from left
                counts[ind(s[left])] -= 1
                if counts[ind(s[left])] < subCounts[ind(s[left])] and not subCounts[ind(s[left])] == 0:
                    have -= 1
                left += 1

            # Expand window to the right
            right += 1
        
        return s[res[0]:res[1]+1] if minLen != N + 1 else ""

In [11]:
minWindow("ADOBECODEBANC", "ABC")

'BANC'